# Toronto Neighborhoods
Notebook for Segmenting and Clustering Neighborhoods in Toronto class

## Task 1: Scrapping data

In [5]:
print('HI')

HI


## Task 2: Dataframe with lat/long

## Task 3: Explore and cluster the neighborhoods in Toronto